In [ ]:
from dotenv import load_dotenv
import locale
import numpy as np
import os
import pandas as pd
import sqlalchemy as sa

load_dotenv()

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

engine = sa.create_engine(os.getenv("URL_MYSQL"))

In [ ]:
minhas_apostas = [
    [ 5, 15, 26, 27, 46, 53],  # aposta n.° 1
    [ 3, 12, 19, 20, 45, 47],  # aposta n.° 2
    [ 1, 10, 17, 41, 42, 56],  # aposta n.° 3
    [ 2, 10, 13, 27, 53, 55],  # aposta n.° 4
    [ 6,  7,  8, 11, 43, 56],  # aposta n.° 5
    [ 8, 10, 14, 25, 33, 34],  # aposta n.° 6
    [ 5, 11, 16, 40, 43, 57],  # aposta n.° 7
    [ 4,  5,  8, 13, 17, 38],  # aposta n.° 8
    [13, 24, 32, 49, 51, 60],  # aposta n.° 9
    [11, 16, 19, 43, 58, 60],  # aposta n.° 10
    [ 3,  5, 10, 20, 35, 46],  # aposta n.° 11
    [ 2,  9, 10, 19, 31, 57],  # aposta n.° 12
    [ 4, 18, 20, 21, 39, 57],  # aposta n.° 13
    [ 2, 11, 22, 36, 49, 60],  # aposta n.° 14
    [ 2, 21, 39, 48, 52, 57],  # aposta n.° 15
    [14, 41, 45, 50, 54, 59],  # aposta n.° 16
    [13, 20, 22, 25, 28, 39],  # aposta n.° 17
    [ 1, 16, 21, 34, 49, 54],  # aposta n.° 18
]

In [ ]:
new_games = [
    {"concurso": 2747, "data": "2024-07-10", "bola1": 14, "bola2": 17, "bola3": 24, "bola4": 28, "bola5": 36, "bola6": 45, "ganhou": False},
]

df_games = pd.DataFrame(new_games)

rows_games = df_games.to_sql(name="megasena", con=engine, if_exists="append", index=False)
print(f"Foram {rows_games} jogo(s) inserido(s) com sucesso!")

In [ ]:
df_last = pd.read_sql(sql=sa.text("SELECT * FROM megasena"), con=engine)

df_last["concurso"] = df_last["concurso"].astype(str).str.zfill(4)
df_last["data"] = pd.to_datetime(df_last["data"]).dt.strftime("%x (%a)")
df_last[df_last.columns[2:-1]] = df_last[df_last.columns[2:-1]].map(lambda x: f"{x:02}")
df_last["ganhou"] = np.where(df_last["ganhou"], "Sim", "Não")
df_last.columns = ["Concurso", "Data", "Bola 1", "Bola 2", "Bola 3", "Bola 4", "Bola 5", "Bola 6", "Ganhou?"]

df_last.tail(15)

In [ ]:
stmt = "SELECT * FROM megasena WHERE data IN (SELECT MAX(data) FROM megasena GROUP BY YEAR(data) HAVING YEAR(data) <> YEAR(CURRENT_DATE))"
df_mega_da_virada = pd.read_sql(sql=sa.text(stmt), con=engine)

df_mega_da_virada["concurso"] = df_mega_da_virada["concurso"].astype(str).str.zfill(4)
df_mega_da_virada["data"] = pd.to_datetime(df_mega_da_virada["data"]).dt.strftime("%x (%a)")
df_mega_da_virada[df_mega_da_virada.columns[2:-1]] = df_mega_da_virada[df_mega_da_virada.columns[2:-1]].map(lambda x: f"{x:02}")
df_mega_da_virada["ganhou"] = np.where(df_mega_da_virada["ganhou"], "Sim", "Não")
df_mega_da_virada.columns = ["Concurso", "Data", "Bola 1", "Bola 2", "Bola 3", "Bola 4", "Bola 5", "Bola 6", "Ganhou?"]

df_mega_da_virada